In [1]:
import openai, os

In [2]:
openai.api_key = os.environ.get('OPENAI_API_KEY')

In [66]:
def query(prompt, **kwargs):
    """
    wrapper for the API to save the prompt and the result
    """
    args = {
    "engine":"text-davinci-003",
    "temperature":0,
    "max_tokens":400,
    "stop":"END",
    }

    for key in kwargs:
        args[key] = kwargs[key]

    r = openai.Completion.create(prompt=prompt, **args)
    return r["choices"][0]["text"].strip()


In [67]:
query('1+1=')

'2\n\n2+2=4'

In [78]:
overviewForStoryPrompt = """ /* Generate an outline for a story

type character{{
  "name": str,
  "profession": str,
  "age": int,
}}

interface outline {{
  "setting": str, //where the section takes place
  "characters": character[], //the character names
  "conflict": str, //what conflict needs to get solved
  "plotTwist": str, //any plot twists to occur
  "resolution": str // how the story concludes
}}

const generatePlot = (theme, genre, style) : outline =>{{
    return ai.generatePlot(theme)
}}


theme = '{}'
genre = '{}'
style = '{}'
//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generatePlot(theme, genre, style))
Out:"""

In [97]:
theme = "civil war"
genre = "movie"
style = "shakesepear"


In [98]:
rawOverview = query(overviewForStoryPrompt.format(theme, genre, style), temperature=.5)

In [99]:
rawOverview

'{\n  "setting": "During the civil war",\n  "characters": [\n    {\n      "name": "John",\n      "profession": "Soldier",\n      "age": 22\n    },\n    {\n      "name": "Jane",\n      "profession": "Nurse",\n      "age": 18\n    },\n    {\n      "name": "Jack",\n      "profession": "General",\n      "age": 55\n    }\n  ],\n  "conflict": "John and Jane must decide whether to fight for their side or flee the war",\n  "plotTwist": "Jack turns out to be John\'s father",\n  "resolution": "John and Jane decide to flee the war and start a new life together"\n}'

In [100]:
import json

In [101]:
parsedOverview = json.loads(rawOverview)

In [102]:
parsedOverview

{'setting': 'During the civil war',
 'characters': [{'name': 'John', 'profession': 'Soldier', 'age': 22},
  {'name': 'Jane', 'profession': 'Nurse', 'age': 18},
  {'name': 'Jack', 'profession': 'General', 'age': 55}],
 'conflict': 'John and Jane must decide whether to fight for their side or flee the war',
 'plotTwist': "Jack turns out to be John's father",
 'resolution': 'John and Jane decide to flee the war and start a new life together'}

In [103]:
outlineForStoryPrompt = """ /* Generate an outline for a story

type section{{
  "name": str, // the name of the section
  "description": str, // a brief overview of what happens in the section
}}

interface outline {{
  "parts": section[] //all of the sections in the story
}}

const generateOutline = (storyOverview) : outline =>{{
    return ai.generateOutline(storyOverview)
}}

overview = '{}'

//make sure to include the keys in quotes so we can json.parse
console.log("Out: " + generateOutline(overview))
Out:"""

In [104]:
rawOutline = query(outlineForStoryPrompt.format(rawOverview), temperature=.5)

In [105]:
rawOutline

'{\n  "parts": [\n    {\n      "name": "Introduction",\n      "description": "Introduce the setting, characters, and conflict of the story."\n    },\n    {\n      "name": "Rising Action",\n      "description": "John and Julia must work together to save the Union from destruction."\n    },\n    {\n      "name": "Plot Twist",\n      "description": "Julia is revealed to be a Confederate spy, but John convinces her to switch sides."\n    },\n    {\n      "name": "Climax",\n      "description": "John and Julia lead the Union to victory, ending the Civil War."\n    },\n    {\n      "name": "Conclusion",\n      "description": "Wrap up the story and reflect on the events that took place."\n    }\n  ]\n}'

In [106]:
parsedOutline = json.loads(rawOutline)

In [107]:
parsedOutline

{'parts': [{'name': 'Introduction',
   'description': 'Introduce the setting, characters, and conflict of the story.'},
  {'name': 'Rising Action',
   'description': 'John and Julia must work together to save the Union from destruction.'},
  {'name': 'Plot Twist',
   'description': 'Julia is revealed to be a Confederate spy, but John convinces her to switch sides.'},
  {'name': 'Climax',
   'description': 'John and Julia lead the Union to victory, ending the Civil War.'},
  {'name': 'Conclusion',
   'description': 'Wrap up the story and reflect on the events that took place.'}]}

config file is an object, inputs and outputs

for each input, that's on the bottom, each output on the top

# WE STARTED TO WRITE THIS THEN REALIZED WE SKIPPING AHEAD

In [108]:
promptGenerator = """Generate typescript files to perform a function, e.g.

Purpose: generate a poem
{{
  "description": "generate a poem",
  "shortName": "generatePoem",
  "inputs": [
  {{"style": "shakespearean",
    "interpretationOfStyle": "the author type to emultae"
  }}
  ],
  "outputs": [
    {{
      "poem": "O Romeo, Romeo, wherefore art thou Romeo?\nDeny thy father and refuse thy name."
    }}
  ]
}}

Purpose: {}
"""

In [110]:
query(promptGenerator.format('generate a paragraph'))

'{\n  "description": "generate a paragraph",\n  "shortName": "generateParagraph",\n  "inputs": [\n  {"style": "formal",\n    "interpretationOfStyle": "the author type to emultae"\n  }\n  ],\n  "outputs": [\n    {\n      "paragraph": "The sun was setting on the horizon, casting a golden hue across the sky. The birds were singing their evening songs, and the air was filled with the scent of flowers. It was a peaceful and tranquil moment, one that would be remembered for years to come."\n    }\n  ]\n}\n\n// Pseudocode\n\n// Generate a Poem\n\n// Function: generatePoem\n// Inputs: style (string), interpretationOfStyle (string)\n// Outputs: poem (string)\n\n// Begin\n\n// Set poem to empty string\n// Set style to input style\n// Set interpretationOfStyle to input interpretationOfStyle\n\n// IF style is "shakespearean"\n  // Set poem to "O Romeo, Romeo, wherefore art thou Romeo?\n  // Deny thy father and refuse thy name."\n// ELSE\n  // Set poem to "The style specified is not supported."\n//